# Attack Password with an Template Attack V

In [ ]:
%run '../util/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Motivation</a></span></li><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#Profiling-phase" data-toc-modified-id="Profiling-phase-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Profiling phase</a></span><ul class="toc-item"><li><span><a href="#Finding-points-of-interest" data-toc-modified-id="Finding-points-of-interest-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Finding points of interest</a></span></li><li><span><a href="#Calculate-template" data-toc-modified-id="Calculate-template-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Calculate template</a></span></li></ul></li><li><span><a href="#Attacking-phase" data-toc-modified-id="Attacking-phase-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Attacking phase</a></span><ul class="toc-item"><li><span><a href="#Capture" data-toc-modified-id="Capture-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Capture</a></span></li><li><span><a href="#Apply-template" data-toc-modified-id="Apply-template-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Apply template</a></span></li></ul></li><li><span><a href="#The-full-attack" data-toc-modified-id="The-full-attack-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>The full attack</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

## Motivation

In the last notebooks we revealed interesting ways of beating the CPA by adding randomness into the loop. But there is another very easy possibility to prohibit a CPA a priori: Punish wrong attempts either by introducing an increasing waiting time between two attempts or by counting wrong attempts and locking out the attacker finally. In both cases CPA looses its point of attack because the number of traces which can be recorded is very limited.

This is the point where Template Attack comes into play. It is divided into two phases: A profiling phase and an attack phase. During the profiling phase a huge number of traces is recorded from an open device which is under full control of the attacker. A so called *template* can be created out of these traces. During the attacking phase only a few traces are needed (even one can be sufficient). The templates is applied on these few traces and gives directly a conclusion about the secret.

As the template attack is described very detailed in a [ChipWhisperer example](https://chipwhisperer.readthedocs.io/en/latest/tutorials/pa_profiling_1-openadc-cwlitexmega.html) only the application of the attack is shown here.

## Basic Setup

Define Variables

In [ ]:
%run "../util/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'simpleserial-passwordcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 500

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
import warnings
import random
import tqdm
import numpy as np

command = '1'
"""Command to execute"""

password_length = 8
"""Number of bytes of password"""

random_length = 32
"""Number of bytes of random input"""

def capture(command, data):
    scope.arm()
    target.simpleserial_write(command, data)

    ret = scope.capture()

    i = 0
    while not target.is_done():
        i += 1
        time.sleep(0.05)
        if i > 100:
            warnings.warn("Target did not finish operation")
            return None

    if ret:
        warnings.warn("Timeout happened during capture")
        return None

    return scope.get_last_trace()

def target_set_random(random_input=None):
    if random_input is not None:
        rand = random_input()
    else:
        rand = bytes(random.choices(range(0, 256), k=random_length))
    target.simpleserial_write('r', rand)
    return rand

def target_set_password(password):
    target.simpleserial_write('p', password)
    return target.simpleserial_read('r', password_length)

def target_check_password(command, password):
    target.simpleserial_write(command, password)
    return bytes(target.simpleserial_read('r', 1))[0] == 0

def capture_random_passwords(command, size=500):
    traces = []
    textins = []
    passwords = []
    for _ in tqdm.tqdm_notebook(range(size)):
        passwords.append(bytes(random.choices(range(0, 256), k=password_length)))
        textins.append(bytes(random.choices(range(0, 256), k=password_length)))
        target_set_password(passwords[-1])
        traces.append(capture(command, textins[-1]))
    return np.array(traces), textins, passwords

def capture_random_attempt(command, size=10):
    traces = []
    textins = []
    for _ in tqdm.tqdm_notebook(range(size)):
        textins.append(bytes(random.choices(range(0, 256), k=password_length)))
        traces.append(capture(command, textins[-1]))
    return np.array(traces), textins

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

def cov(x, y):
    return np.cov(x, y)[0][1]

## Profiling phase

In [ ]:
traces, textins, passwords = capture_random_passwords(command=command, size=30000)

### Finding points of interest

In [ ]:
# Groupy traces by different hamming weights
traces_per_hw = [[] for _ in range(9)]
for trace, passwd, textin in zip(traces, passwords, textins):
    traces_per_hw[hw(passwd[0] ^ textin[0])].append(trace)
traces_per_hw = list(map(np.array, traces_per_hw))
means_per_hw = [np.average(trace, 0) for trace in traces_per_hw]

# Calculate sum of absolute differences
diffs = np.zeros(means_per_hw[0].shape[0])
for mean_i in means_per_hw:
    for mean_j in means_per_hw:
        diffs += np.abs(mean_i - mean_j)

# Plot
p = figure()
p.add_tools(CrosshairTool())
p.line(range(len(diffs)), diffs, color='blue', legend='Sum of diffs')
show(p)

# Take highest 10 peaks as pois
pois = np.argsort(diffs)[-5:]

### Calculate template

In [ ]:
template_mean = np.zeros((9, len(pois)))
template_cov = np.zeros((9, len(pois), len(pois)))

for ham in range(template_mean.shape[0]):
    for i in range(len(pois)):
        template_mean[ham, i] = means_per_hw[ham][pois[i]]
        for j in range(len(pois)):
            template_cov[ham, i, j] = cov(
                traces_per_hw[ham][:, pois[i]],
                traces_per_hw[ham][:, pois[j]],
            )
print('template_mean', template_mean.shape, '=\n', template_mean)
print('template_cov', template_cov.shape, '=\n', template_cov)

## Attacking phase

### Capture

In [ ]:
target_set_password(b'ifx2019a')
traces_attack, textins_attack = capture_random_attempt(command, size=10)

### Apply template

In [ ]:
from scipy.stats import multivariate_normal

trylist='abcdefghijklmnopqrstuvwxyz0123456789'

# 2: Attack
# Running total of log P_k
P_k = np.zeros(len(trylist))
for j in range(len(traces_attack)):
    # Grab key points and put them in a small matrix
    a = [traces_attack[j][pois[i]] for i in range(len(pois))]
    
    # Test each key
    for k in range(len(trylist)):
        # Find HW coming out of sbox
        guess_hw = hw(textins_attack[j][0] ^ ord(trylist[k]))
    
        # Find p_{k,j}
        rv = multivariate_normal(template_mean[guess_hw], template_cov[guess_hw])
        p_kj = rv.logpdf(a)
   
        # Add it to running total
        P_k[k] += p_kj

    # Print our top 5 results so far
    # Best match on the right
    print(j, 'best five: ', ' '.join([trylist[j] for j in P_k.argsort()[-5:][::-1]]))
    
guess = P_k.argsort()[-1]
print(hex(guess))

## The full attack

In [ ]:
from scipy.stats import multivariate_normal

def generate_template_for_index(traces, textins, passwords, index, poilength):
    # Groupy traces by different hamming weights
    traces_per_hw = [[] for _ in range(9)]
    for trace, passwd, textin in zip(traces, passwords, textins):
        traces_per_hw[hw(passwd[index] ^ textin[index])].append(trace)
    traces_per_hw = list(map(np.array, traces_per_hw))
    means_per_hw = [np.average(trace, 0) for trace in traces_per_hw]

    # Calculate sum of absolute differences
    diffs = np.zeros(means_per_hw[0].shape[0])
    for mean_i in means_per_hw:
        for mean_j in means_per_hw:
            diffs += np.abs(mean_i - mean_j)

    # Take highest peaks as pois
    pois = np.argsort(diffs)[-poilength:]
    
    template_mean = np.zeros((9, len(pois)))
    template_cov = np.zeros((9, len(pois), len(pois)))

    for ham in range(template_mean.shape[0]):
        for i in range(len(pois)):
            template_mean[ham, i] = means_per_hw[ham][pois[i]]
            for j in range(len(pois)):
                template_cov[ham, i, j] = cov(
                    traces_per_hw[ham][:, pois[i]],
                    traces_per_hw[ham][:, pois[j]],
                )
    return {
        'rv': [multivariate_normal(template_mean[ham], template_cov[ham]) for ham in range(9)], 
        'pois': pois
    }

def generate_template(traces, textins, passwords, poilength=5):
    return [generate_template_for_index(traces, textins, passwords, i, poilength) for i in range(len(textins[0]))]

In [ ]:
def attack_with_template(
    traces, 
    textins, 
    template,
    trylist='abcdefghijklmnopqrstuvwxyz0123456789',
):
    password_guess = []
    for password_index in range(len(textins[0])):
        P_k = np.zeros(len(trylist))
        for j in range(len(traces)):
            a = [traces[j][template[password_index]['pois'][i]] for i in range(len(template[password_index]['pois']))]
            for guess_index in range(len(trylist)):
                guess_hw = hw(textins[j][password_index] ^ ord(trylist[guess_index]))
                p_kj = template[password_index]['rv'][guess_hw].logpdf(a)
                P_k[guess_index] += p_kj

        password_guess.append(trylist[P_k.argsort()[-1]])
    return password_guess

In [ ]:
#traces, textins, passwords = capture_random_passwords(command=command, size=30000)
template = generate_template(traces, textins, passwords, poilength=3)

In [ ]:
target_set_password(b'ifx2019a')
attack_with_template(*capture_random_attempt(command, size=20), template)

## Disconnect

In [ ]:
scope.dis()
target.dis()